# TO DO TO RUN
1. You need to add the `.item()` for `words.append(idx_to_tok[idx.item()])` as you see here. It is in the `seq_to_string` for `utils.py`. May need to restart kernel to get changes to import

2. Add the `data.pkl` file to the directory

3. Go to [here](https://github.com/adamklec/copynet.git) for more info

In [1]:
! git clone https://github.com/zachary-m/copynet.git
! git clone https://github.com/zachary-m/gc.git
! pip install tensorboardX

Cloning into 'copynet'...
remote: Enumerating objects: 36, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 36 (delta 2), reused 0 (delta 0), pack-reused 30
Unpacking objects: 100% (36/36), done.
Cloning into 'gc'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 3 (delta 0), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (3/3), done.
     |████████████████████████████████| 122kB 17.7MB/s 


In [2]:
import os 
print(os.getcwd())
os.chdir("/content/copynet/")
print(os.getcwd())
if not os.path.exists("/content/copynet/data"):
    os.makedirs("/content/copynet/data")

/content
/content/copynet


In [3]:
!nvidia-smi

Sun May  2 20:42:42 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   53C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
# torch.cuda.empty_cache()

In [5]:
import argparse
import time
import numpy as np
import pandas as pd
import torch
import string
from torch import optim
from torch.autograd import Variable
from torch.utils.data import DataLoader, Dataset
import re
from dataset import SequencePairDataset
from model.encoder_decoder import EncoderDecoder
from evaluate import evaluate
from utils import to_np, trim_seqs

from tensorboardX import SummaryWriter
from tqdm.notebook import tqdm
from nltk.translate.bleu_score import corpus_bleu, SmoothingFunction

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if torch.cuda.is_available():
    print("Using the GPU!")
else:
    print("WARNING: Could not find GPU! Using CPU only")

Using the GPU!


In [6]:
df = pd.read_pickle("/content/gc/train_test_val.pkl")
print(df.data_type.unique())
df.head()

['test' 'train' 'val']


,src,target,data_type
0,ויהיו שם שדי מואב ויבאו יהודה לחם מבית אפרתים ...,שם ונשארו מואב לארץ באו הם שביהודה לחם בית היא...,test
1,התבן בהיות כאשר ביומו דבר יום מעשיכם כלו לאמר ...,הקש את שקבלתם בזמן יום בכל שעשיתם כמו מעשיכם א...,test
2,ומש וגתר וחול עוץ ארם ובני,ומש וגתר וחול עוץ ארם ובני,test
3,סביב המזבח על קרנות ונתן השעיר ומדם הפר מדם ול...,מסביב המזבח פנות על וישים התיש ומדם הפר מדם וי...,test
4,תעוננו ולא תנחשו על הדם תאכלו,עינים תאחזו ואל תכשפו אל הדם עם בעודו בשר תאכל...,test


In [7]:

# def remove_notes(word_list):
#   return " ".join([re.sub(r'[\u0591-\u05BD\u05BF-\u05C2\u05C4-\u05C7]', '', token) for token in word_list])

# df["src"] = df["src"].apply(lambda row: remove_notes(row))

# df["target"] = df["target"].apply(lambda row: remove_notes(row))
# results = ['א', 'ב', 'ג', 'ד', 'ה', 'ו', 'ז', 'ח', 'ט', 'י', 'ך',  'כ',  'ל',  'ם',  'מ','ן', 'נ','ס', 'ע', 'ף', 'פ', 'ץ', 'צ', 'ק', 'ר', 'ש', 'ת']
# map_letters = dict(list(zip(results, string.ascii_lowercase + "@")))
# # df = df.replace(map_letters, regex=True)

# df["reversed_src"] = df["src"].apply(lambda row: row[::-1])
# df["reversed_target"] = df["target"].apply(lambda row: row[::-1])

In [8]:
df

,src,target,data_type
0,ויהיו שם שדי מואב ויבאו יהודה לחם מבית אפרתים ...,שם ונשארו מואב לארץ באו הם שביהודה לחם בית היא...,test
1,התבן בהיות כאשר ביומו דבר יום מעשיכם כלו לאמר ...,הקש את שקבלתם בזמן יום בכל שעשיתם כמו מעשיכם א...,test
2,ומש וגתר וחול עוץ ארם ובני,ומש וגתר וחול עוץ ארם ובני,test
3,סביב המזבח על קרנות ונתן השעיר ומדם הפר מדם ול...,מסביב המזבח פנות על וישים התיש ומדם הפר מדם וי...,test
4,תעוננו ולא תנחשו על הדם תאכלו,עינים תאחזו ואל תכשפו אל הדם עם בעודו בשר תאכל...,test
...,...,...,...
9217,ישן תאכלו תבואתה עד בוא התשיעת השנה ישן מן התב...,הישנה מהתבואה תאכלו שנה אותה תבואת הגעת התשיעי...,train
9218,במועדם אתם אשר תקראו קדש מקראי יהוה מועדי אלה,בזמנם אותם שתקבעו קודש מאורעות יהוה חגי אלה,train
9219,האחד לאיל עשרנים שני לפר עשרנים שלשה בשמן בלול...,איל לכל ליטר כ6 5 האיפה עשיריות שלוש בשמן מערב...,train
9220,הישראלי ואיש הישראלית בן במחנה וינצו ישראל בני...,במחנה רבו ישראלי ואיש הישראלית של הבן ישראל בנ...,train


In [9]:
vals = {"test":0, "val":0, "train":0}
train_val_test = {"test":[], "val":[], "train":[]}
for idx, (i, row) in enumerate(df.iterrows()): # copy path if copy
    # print(row["data_type"])
    path = "/content/copynet/data/{}{}.txt".format(row["data_type"], vals[row["data_type"]])
    row[["src",	"target"]].to_csv(path, index=False, header=False)
    train_val_test[row["data_type"]].append(path)
    vals[row["data_type"]] += 1
    

In [10]:
class CustomDataSet(Dataset):
  """
  The Class will act as the container for our dataset. It will take your dataframe, the root path, and also the transform function for transforming the dataset.
  """
  def __init__(self, file_name_list, transform=None):
    self.file_name_list = file_name_list
    self.transform = transform
  
  def __len__(self):
      # Return the length of the dataset
    return len(self.file_name_list)
  
  def __getitem__(self, idx):
      # Return the observation based on an index. Ex. dataset[0] will return the first element from the dataset, in this case the image and the label.
    if torch.is_tensor(idx):
        idx = int(idx.item())
    file_path = self.file_name_list[idx]
    input_val, target_val = pd.read_csv(file_path, header=None).T.values.ravel()
    return (input_val, target_val)

In [11]:
writer = SummaryWriter()

def seq_to_string(seq, idx_to_tok, input_tokens=None):
    # print("DO", seq, idx_to_tok)
    # print("HHHHHHHHHHHHHHHHHHHHHHHHHHHHHH", flush=True)
    vocab_size = len(idx_to_tok)
    seq_length = (seq != 0).sum()
    words = []
    # print("HHELLO")
    for idx in seq[:seq_length]:
        
        # print(idx, flush=True)
        # print(idx.item(), flush=True)
        # print("HEELLO", flush=True)
        if idx < vocab_size:
            
            words.append(idx_to_tok[idx.item()]) # CHANGE
        elif input_tokens is not None:
            words.append(input_tokens[idx - vocab_size])
        else:
            words.append('<???>')
    string = ' '.join(words)
    return string
def train(encoder_decoder: EncoderDecoder,
          train_data_loader: DataLoader,
          model_name,
          val_data_loader: DataLoader,
          keep_prob,
          teacher_forcing_schedule,
          lr,
          max_length):

    global_step = 0
    loss_function = torch.nn.NLLLoss(ignore_index=0)
    optimizer = optim.Adam(encoder_decoder.parameters(), lr=lr)
    model_path = './model/' + model_name + '/'

    for epoch, teacher_forcing in enumerate(teacher_forcing_schedule):
        print('epoch %i' % epoch)

        for batch_idx, (input_idxs, target_idxs, input_tokens, target_tokens) in enumerate(tqdm(train_data_loader)):
            # input_idxs and target_idxs have dim (batch_size x max_len)
            # they are NOT sorted by length
            # print(input_tokens, target_tokens)
            lengths = (input_idxs != 0).long().sum(dim=1)
            sorted_lengths, order = torch.sort(lengths, descending=True)

            input_variable = Variable(input_idxs[order, :][:, :max(lengths)])
            target_variable = Variable(target_idxs[order, :])
            # print(target_variable)

            optimizer.zero_grad()
            output_log_probs, output_seqs = encoder_decoder(input_variable,
                                                            list(sorted_lengths),
                                                            targets=target_variable,
                                                            keep_prob=keep_prob,
                                                            teacher_forcing=teacher_forcing)
            # print(output_seqs)
            batch_size = input_variable.shape[0]

            flattened_outputs = output_log_probs.view(batch_size * max_length, -1)
            # print(flattened_outputs)

            batch_loss = loss_function(flattened_outputs, target_variable.contiguous().view(-1))
            batch_loss.backward()
            optimizer.step()

            batch_outputs = trim_seqs(output_seqs)
            # print("target_variable", target_variable)
            batch_targets = [[list(seq[seq > 0])] for seq in list(to_np(target_variable))]
            # print("batch targets", batch_targets)
            batch_bleu_score = corpus_bleu(batch_targets, batch_outputs, smoothing_function=SmoothingFunction().method1)
            # print("INPUT STRING")
            # print("train loss {}".format(batch_loss), "train BLEU {}".format(batch_bleu_score), )
            # WHAT IS THIS?????????
            if global_step < 10 or (global_step % 10 == 0 and global_step < 100) or (global_step % 100 == 0 and epoch < 2):
                # print("INPUT STRING 2", input_tokens)
                input_string = "Amy, Please schedule a meeting with Marcos on Tuesday April 3rd. Adam Kleczewski"
                
                output_string = encoder_decoder.get_response(input_string)
                writer.add_text('schedule', output_string, global_step=global_step)

                input_string = "Amy, Please cancel this meeting. Adam Kleczewski"
                output_string = encoder_decoder.get_response(input_string)
                writer.add_text('cancel', output_string, global_step=global_step)

            if global_step % 100 == 0:

                writer.add_scalar('train_batch_loss', batch_loss, global_step)
                writer.add_scalar('train_batch_bleu_score', batch_bleu_score, global_step)

                for tag, value in encoder_decoder.named_parameters():
                    tag = tag.replace('.', '/')
                    writer.add_histogram('weights/' + tag, value, global_step, bins='doane')
                    writer.add_histogram('grads/' + tag, to_np(value.grad), global_step, bins='doane')

            global_step += 1

        val_loss, val_bleu_score = evaluate(encoder_decoder, val_data_loader)

        writer.add_scalar('val_loss', val_loss, global_step=global_step)
        writer.add_scalar('val_bleu_score', val_bleu_score, global_step=global_step)


        # for i in val_data_loader:
        #   print(i)
        encoder_embeddings = encoder_decoder.encoder.embedding.weight.data
        encoder_vocab = encoder_decoder.lang.tok_to_idx.keys()
        writer.add_embedding(encoder_embeddings, metadata=encoder_vocab, global_step=0, tag='encoder_embeddings')

        decoder_embeddings = encoder_decoder.decoder.embedding.weight.data
        decoder_vocab = encoder_decoder.lang.tok_to_idx.keys()
        writer.add_embedding(decoder_embeddings, metadata=decoder_vocab, global_step=0, tag='decoder_embeddings')

    
        input_string = "Amy, Please schedule a meeting with Marcos on Tuesday April 3rd. Adam Kleczewski"
        output_string = encoder_decoder.get_response(input_string)
        writer.add_text('schedule', output_string, global_step=global_step)

        input_string = "Amy, Please cancel this meeting. Adam Kleczewski"
        output_string = encoder_decoder.get_response(input_string)
        writer.add_text('cancel', output_string, global_step=global_step)

        print('val loss: %.5f, val BLEU score: %.5f' % (val_loss, val_bleu_score))
        torch.save(encoder_decoder, "%s%s_%i.pt" % (model_path, model_name, epoch))

        print('-' * 100)


def main(model_name, use_cuda, batch_size, teacher_forcing_schedule, keep_prob, val_size, lr, decoder_type, vocab_limit, hidden_size, embedding_size, max_length, seed=42):

    model_path = './model/' + model_name + '/'

    # TODO: Change logging to reflect loaded parameters

    print("training %s with use_cuda=%s, batch_size=%i"% (model_name, use_cuda, batch_size))
    print("teacher_forcing_schedule=", teacher_forcing_schedule)
    print("keep_prob=%f, val_size=%f, lr=%f, decoder_type=%s, vocab_limit=%i, hidden_size=%i, embedding_size=%i, max_length=%i, seed=%i" % (keep_prob, val_size, lr, decoder_type, vocab_limit, hidden_size, embedding_size, max_length, seed), flush=True)

    if os.path.isdir(model_path):

        print("loading encoder and decoder from model_path")
        encoder_decoder = torch.load(model_path + model_name + '.pt')

        print("creating training and validation datasets with saved languages")
        train_dataset = SequencePairDataset(lang=encoder_decoder.lang,
                                            use_cuda=use_cuda,
                                            is_val=False,
                                            val_size=val_size,
                                            use_extended_vocab=(encoder_decoder.decoder_type=='copy'))
        
        val_dataset = SequencePairDataset(lang=encoder_decoder.lang,
                                          use_cuda=use_cuda,
                                          is_val=True,
                                          val_size=val_size,
                                          use_extended_vocab=(encoder_decoder.decoder_type=='copy'))

    else:
        os.mkdir(model_path)

        print("creating training and validation datasets")
        train_dataset = SequencePairDataset(vocab_limit=vocab_limit,
                                            use_cuda=use_cuda,
                                            is_val=False,
                                            val_size=val_size,
                                            seed=seed,
                                            use_extended_vocab=(decoder_type=='copy'))
        print("TRAIN SIZE", len(train_dataset))
        val_dataset = SequencePairDataset(lang=train_dataset.lang,
                                          use_cuda=use_cuda,
                                          is_val=True,
                                          val_size=val_size,
                                          seed=seed,
                                          use_extended_vocab=(decoder_type=='copy'))
        print("TEST SIZE", len(val_dataset))
        print("creating encoder-decoder model")
        encoder_decoder = EncoderDecoder(train_dataset.lang,
                                         max_length,
                                         embedding_size,
                                         hidden_size,
                                         decoder_type)

        torch.save(encoder_decoder, model_path + '/%s.pt' % model_name)

    if use_cuda:
        encoder_decoder = encoder_decoder.to(device)
    else:
        encoder_decoder = encoder_decoder.cpu()

    train_data_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_data_loader = DataLoader(val_dataset, batch_size=batch_size)

    train(encoder_decoder,
          train_data_loader,
          model_name,
          val_data_loader,
          keep_prob,
          teacher_forcing_schedule,
          lr,
          encoder_decoder.decoder.max_length)



In [12]:
# vocab_limit = []
# for i, row in df.iterrows():
# https://github.com/omilab/Neural-Sentiment-Analyzer-for-Modern-Hebrew/tree/master/data

In [13]:
model = main(model_name = "models",
     use_cuda=True,
     batch_size=16,
     teacher_forcing_schedule= [.001]*5 + [.0005]*5+ [.0001]*10+ [.00001]*5,  #[.001]*5 + [.0005]*10 + [.001]*15 +
     keep_prob=1,
     val_size=0.1,
     lr=0.0001,
     decoder_type='copy',
     vocab_limit=5000,
     hidden_size=500, # 500 https://arxiv.org/pdf/1603.06393.pdf  # increase
     embedding_size=350, #350  increase
     max_length=500, # increase
     seed=42)

training models with use_cuda=True, batch_size=16
teacher_forcing_schedule= [0.001, 0.001, 0.001, 0.001, 0.001, 0.0005, 0.0005, 0.0005, 0.0005, 0.0005, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 1e-05, 1e-05, 1e-05, 1e-05, 1e-05]
keep_prob=1.000000, val_size=0.100000, lr=0.000100, decoder_type=copy, vocab_limit=5000, hidden_size=500, embedding_size=350, max_length=500, seed=42
creating training and validation datasets
reading file 0/8300
reading file 1000/8300
reading file 2000/8300
reading file 3000/8300
reading file 4000/8300
reading file 5000/8300
reading file 6000/8300
reading file 7000/8300
reading file 8000/8300
TRAIN SIZE 8300
TEST SIZE 922
creating encoder-decoder model
epoch 0


ValueError: ignored

In [ ]:
# /content/copynet/evaluate.py:28: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
#   input_variable = Variable(input_idxs[order, :][:, :max(input_lengths)], volatile=True)
# /content/copynet/evaluate.py:29: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
#   target_variable = Variable(target_idxs[order, :], volatile=True)

In [ ]:
if __name__ == '__main__':
    parser = argparse.ArgumentParser(description='Parse training parameters')
    parser.add_argument('./data/', type=str,
                        help='the name of a subdirectory of ./model/ that '
                             'contains encoder and decoder model files')
    
    parser.add_argument('--epochs', type=int, default=10,
                        help='the number of epochs to train')

    parser.add_argument('--use_cuda', action='store_true',
                        help='flag indicating that cuda will be used')

    parser.add_argument('--batch_size', type=int, default=128,
                        help='number of examples in a batch')

    parser.add_argument('--teacher_forcing_fraction', type=float, default=0.5,
                        help='fraction of batches that will use teacher forcing during training')

    parser.add_argument('--scheduled_teacher_forcing', action='store_true',
                        help='Linearly decrease the teacher forcing fraction '
                             'from 1.0 to 0.0 over the specified number of epocs')

    parser.add_argument('--keep_prob', type=float, default=1.0,
                        help='Probablity of keeping an element in the dropout step.')

    parser.add_argument('--val_size', type=float, default=0.1,
                        help='fraction of data to use for validation')

    parser.add_argument('--lr', type=float, default=0.001,
                        help='Learning rate.')

    parser.add_argument('--decoder_type', type=str, default='copy',
                        help="Allowed values 'copy' or 'attn'")

    parser.add_argument('--vocab_limit', type=int, default=5000,
                        help='When creating a new Language object the vocab'
                             'will be truncated to the most frequently'
                             'occurring words in the training dataset.')

    parser.add_argument('--hidden_size', type=int, default=256,
                        help='The number of RNN units in the encoder. 2x this '
                             'number of RNN units will be used in the decoder')

    parser.add_argument('--embedding_size', type=int, default=128,
                        help='Embedding size used in both encoder and decoder')

    parser.add_argument('--max_length', type=int, default=200,
                        help='Sequences will be padded or truncated to this size.')

    args = parser.parse_args()

    writer = SummaryWriter('./logs/%s_%s' % (args.model_name, str(int(time.time()))))
    if args.scheduled_teacher_forcing:
        schedule = np.arange(1.0, 0.0, -1.0/args.epochs)
    else:
        schedule = np.ones(args.epochs) * args.teacher_forcing_fraction

    # main(args.model_name, args.use_cuda, args.batch_size, schedule, args.keep_prob, args.val_size, args.lr, args.decoder_type, args.vocab_limit, args.hidden_size, args.embedding_size, args.max_length)
    # main(str(int(time.time())), args.use_cuda, args.batch_size, schedule, args.keep_prob, args.val_size, args.lr, args.decoder_type, args.vocab_limit, args.hidden_size, args.embedding_size, args.max_length)
